In [ ]:
from util import get_freq
import matplotlib.pyplot as plt
import numpy as np
from random import uniform
from NPSmethods import readInImages
from NPSmethods import pupilFunc
from numpy.fft import ifftshift, irfft2
import pickle as pkl
import matplotlib.cm as cm
from komm import AWGNChannel

In [ ]:
#Geranate random assortment of parameters
def make_Paras(seedParas, numToGen):
    outPara = np.empty(shape =(numToGen, len(seedParas)))
    paraTemp = [None] * len(seedParas)
    for i in range(numToGen):
        if i == 0:
            paraTemp = seedParas.copy()
        else:
            for j in range(len(seedParas)):
                para = seedParas[j]
                if j ==3 or j == 5 or j == 6:
                    para = para * uniform(.5, 2)
                paraTemp[j] = para
        outPara[i] = paraTemp
    return outPara
        

#Generate M2k_Fit
def make_M2k_Fit(paras, imgSysData):
    
    A, tau, S0, alpha, phi, beta, delta_s = paras
    _, _, K_X, K_Y = get_freq(imgSysData["CCDPixelSize"],
                                      imgSysData["magnification"], (100,100))
    d = imgSysData["wavelen"] / (2*np.pi*imgSysData["NA"])
    R_p, Theta_p = np.abs(K_X + 1j*K_Y) * d, np.angle(K_X + 1j*K_Y)
    p1 = pupilFunc(R_p, Theta_p + np.pi, tau, S0, alpha, phi, beta)
    p2 = np.conj(pupilFunc(R_p, Theta_p, tau, S0, alpha, phi, beta)) * \
            np.exp(-2*1j*delta_s)
    PSF = (p1 + p2) / (2 * np.cos(delta_s))
    M2k = np.abs(PSF)**2
    M2k_Fit = A * M2k
    M2k_Fit[M2k_Fit.shape[0]//2, M2k_Fit.shape[1]//2] = 0
    
    return M2k_Fit

#Generate NPS noise
def generate_Noise(M2k_Fit, idealOD, numIms):
#     M2k_Fit *= idealOD.sum()
    M2k_Fit = M2k_Fit.flatten()
    NPSs = np.empty(shape = (numIms, len(M2k_Fit)))
    noise_mult = range(1, 1+numIms, 1)
    signal_region = np.where(M2k_Fit != 0)
    noise_region = np.where(M2k_Fit == 0)
    NSR = np.random.normal(loc = 0.1, scale = 0.2, size = len(signal_region[0]))
    bkg_noise = np.random.normal(loc = 0.00695, scale = 0.00157, size = len(noise_region[0]))
    

    for i in range(numIms):
        for j in range(len(signal_region[0])):
            NPSs[i][signal_region[0][j]] = M2k_Fit[signal_region[0][j]] + M2k_Fit[signal_region[0][j]] * NSR[j] / noise_mult[i]
        for k in range(len(noise_region[0])):
            NPSs[i][noise_region[0][k]] = bkg_noise[k]

    NPSs = np.reshape(NPSs, (numIms, 100, 100))
    
    return NPSs

#Turn one NPS to one OD
def NPS_to_ODs(NPSs):
    fakeODs = np.empty_like(NPSs[0])
    
    for NPS in NPSs:
        NPS = np.sqrt(np.abs(NPS))
        noise = ifftshift(irfft2(ifftshift(NPS), s = np.shape(NPSs[0])))
        fakeODs = np.dstack((fakeODs, np.exp(-(noise + idealOD))))

    fakeODs = fakeODs.T
    fakeODs = np.delete(fakeODs, 0, 0)
    
    return fakeODs

In [ ]:
folderpath = './raw_image/141902/'
savepath = './results/141902_101-123/'
seedParas = [0.35060326843236184, 0.6484441665068852, -2.63564159087613, 0.5094606166480791, -1.8773441762223027, 0.8206242586655179, 1.0257364487180909]

numNPSs = 3
numODs = 50
numParas = 1

NPSs = np.empty(shape = (numNPSs * numParas, 100, 100))
fakeODs = np.empty(shape = (numNPSs, 100, 100))
savedParas = np.empty(shape = (numNPSs, 7))

_, idealOD, _, _, _, _ = readInImages(folderpath, 23, 0, trapRegion = (slice(100, 200), slice(100, 200)), noiseRegion = (slice(0, 300), slice(0, 300)))

imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
} 

i = 0
for paras in make_Paras(seedParas, numParas):
    print(i+1)
    NPSs[i * numNPSs:(i + 1) * numNPSs] = generate_Noise(make_M2k_Fit(paras, imgSysData), idealOD, numNPSs)
    tempParas = np.ones(shape = (numNPSs, len(paras))) * paras
    savedParas = np.vstack((savedParas, tempParas))
    i += 1
# NPSs = NPSs[numNPSs:]
savedParas = savedParas[numNPSs:]
fakeODs = np.vstack((fakeODs, NPS_to_ODs(NPSs)))
fakeODs = fakeODs[numNPSs:]

In [ ]:
print(np.shape(NPSs))
print(np.shape(fakeODs))
print(np.shape(savedParas))

In [ ]:
print(seedParas)
print(savedParas[0])
print(savedParas[2])
print(savedParas[0])

In [ ]:
K_x, K_y, _, _ = get_freq(imgSysData["CCDPixelSize"], imgSysData["magnification"], (100,100))
fig = plt.figure(figsize=(5.5, 4.5))
ax = fig.add_subplot(111)
print(np.shape(NPSs[0]))
im = ax.pcolor(K_x, K_y, NPSs[1], cmap=cm.jet, vmin = 0, vmax = .285)
plt.colorbar(im)

In [ ]:
plt.imshow(fakeODs[0])

In [ ]:
pkl.dump(NPSs, open(savepath + "fakeNPSs", "wb"))
pkl.dump(fakeODs, open(savepath + "fakeODs", "wb"))
pkl.dump(savedParas, open(savepath + "savedParas", "wb"))